In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import glob
plt.rcParams.update({'font.size': 14})
import os

import re

In [2]:
EQUATIONS = [
    'wave', 
    'gas_dynamics', 
    'reaction_diffusion', 
    'kuramoto_sivashinsky', 
    'advection',
    'burgers',
]
EQUATION_REAL_NAMES = {
    'wave': 'Wave', 
    'gas_dynamics': 'Gas Dynamics', 
    'reaction_diffusion': 'Reaction-Diffusion', 
    'kuramoto_sivashinsky': 'Kuramoto-Sivashinsky', 
    'advection': 'Advection',
    'burgers': 'Burgers',

}
MODELS = [
    'feast', 
    'gat', 
    'gcn', 
    'point_gnn', 
    'point_transformer', 
    'kernelNN', 
    'graphpde', 
    'persistence',
    'cnn', 
    'resnet', 
    'neuralpde'
]
MODEL_REAL_NAMES = {
    'feast': 'FeaSt', 
    'gat': 'GAT', 
    'gcn': 'GCN', 
    'point_gnn': 'PointGNN', 
    'point_net': 'PointNet',
    'point_transformer': 'Point Transformer', 
    'kernelNN': 'KernelNN', 
    'graphpde': 'GraphPDE', 
    'persistence': 'Persistence',
    'cnn': 'CNN', 
    'resnet': 'ResNet', 
    'neuralpde': 'NeuralPDE'
}

TASK = "forecast"
SUPPORTS = ["cloud", "grid"]
NUM_POINTS = "high"
METRIC = [f"test_rollout_{i}" for i in range(1,17)]

listing = glob.glob("../output/csv_logs/*/high/*/*/*/metrics.csv")
rows = []
for file in listing:
    path = file.split("/")
    equation = path[5]
    model = path[6]
    csv = pd.read_csv(file)[[f"test_rollout_{i}" for i in range(1, 17)]].dropna()
    csv['equation'] = EQUATION_REAL_NAMES[equation]
    csv['model'] = MODEL_REAL_NAMES[model]
    csv = csv.set_index(['equation', 'model'])
    rows.append(csv)

results = pd.concat(rows)
results = results.query("model != \"PointNet\"")


In [ ]:
results

In [5]:
results_single = results.pivot_table(values="test_rollout_16", index="model", columns="equation")
t = results_single.to_latex(float_format='%.2E')
t = results_single.to_markdown()
print(t)

| model             |       Advection |   Burgers |   Gas Dynamics |   Kuramoto-Sivashinsky |   Reaction-Diffusion |     Wave |
|:------------------|----------------:|----------:|---------------:|-----------------------:|---------------------:|---------:|
| CNN               |     0.00161331  |  0.554554 |       0.995382 |            1.26011     |          0.0183483   | 0.561433 |
| FeaSt             |     1.48288     |  0.561197 |       0.819594 |            3.74448     |          0.130149    | 1.61066  |
| GAT               | 41364.1         |  0.833353 |       1.21436  |            5.68925     |          3.85506     | 2.38418  |
| GCN               |     3.51453e+13 | 13.0876   |       7.20633  |            1.70612e+24 |          1.75955e+07 | 7.89253  |
| GraphPDE          |     1.07953     |  0.729879 |       0.969208 |            2.1044      |          0.0800235   | 1.02586  |
| KernelNN          |     0.897431    |  0.72716  |       0.854015 |            2.00334     |          0

/tmp/ipykernel_16241/4010742912.py:2: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  t = results_single.to_latex(float_format='%.2E')


In [72]:
scores = re.findall("\d\D\d\dE\D\d\d", t)
for s in scores:
    exponent = s[6:] if s[6] != "0" else s[7]
    sign = "" if s[5] == "+" else "-"
    formated_s = f"${s[:4]}\cdot 10^{{{sign}{exponent}}}$"
    t = t.replace(s, formated_s)

print(t)

\begin{tabular}{lrrrrrr}
\toprule
equation &  Advection &  Burgers &  Gas Dynamics &  Kuramoto-Sivashinsky &  Reaction-Diffusion &     Wave \\
model             &            &          &               &                       &                     &          \\
\midrule
CNN               &   $5.31\cdot 10^{-5}$ & $1.11\cdot 10^{-2}$ &      $4.20\cdot 10^{-3}$ &              $6.70\cdot 10^{-4}$ &            $3.69\cdot 10^{-4}$ & $1.43\cdot 10^{-3}$ \\
FeaSt             &   $1.30\cdot 10^{-4}$ & $1.16\cdot 10^{-2}$ &      $1.62\cdot 10^{-2}$ &              $1.18\cdot 10^{-2}$ &            $4.89\cdot 10^{-4}$ & $5.23\cdot 10^{-3}$ \\
GAT               &   $9.60\cdot 10^{-3}$ & $4.40\cdot 10^{-2}$ &      $3.75\cdot 10^{-2}$ &              $6.67\cdot 10^{-2}$ &            $9.15\cdot 10^{-3}$ & $1.51\cdot 10^{-2}$ \\
GCN               &   $2.64\cdot 10^{-2}$ & $1.39\cdot 10^{-1}$ &      $8.43\cdot 10^{-2}$ &              $4.37\cdot 10^{-1}$ &            $1.65\cdot 10^{-1}$ & $3.82\cdot 10^{-2

In [74]:
UPPER_LIMIT_DICT = {
    'Wave': 3.1, 
    'Gas Dynamics': 2.1, 
    'Reaction-Diffusion': 4.1, 
    'Kuramoto-Sivashinsky': 4.1, 
    'Advection': 2.6,
    'Burgers': 2.1
}

for equation in EQUATIONS:
    equation = EQUATION_REAL_NAMES[equation]

    for model in MODELS:
        model_name = MODEL_REAL_NAMES[model]
        values = results.query(f"model == \"{model_name}\" and equation == \"{equation}\"").values.flatten()

        if model_name in ["CNN", "ResNet", "NeuralPDE"]:
            linestyle = "dotted"
        else:
            linestyle = "solid"

        if model_name == "Persistence":
            color = "black"
            linewidth = 3
            linestyle = "dashdot"
        else:
            color = None
            linewidth = 2.5
        
        #print(equation, model)
        #print(values)
        plt.plot(range(1, 17), values, label=model_name, linestyle=linestyle, linewidth=linewidth, c=color)
    

    plt.legend()
    plt.ylim(-0.1, UPPER_LIMIT_DICT[equation])
    plt.ylabel("MSE")
    plt.xlabel("Rollout step")
    plt.tight_layout()
    plt.savefig(f"../figures/results/{equation}.pdf", dpi=200, bbox_inches="tight")
    plt.clf()

<Figure size 640x480 with 0 Axes>